In [ ]:
using Oscar

# Worksheet 4: Deformations of IHS, ICIS, and beyond

## Versal deformations of isolated hypersurface singularities

We can consider an isolated hypersurface singularity $f \colon (\mathbb C^n, 0) \to (\mathbb C,0)$ 
up to right- ($\mathcal R$) or contact-equivalence ($\mathcal K)$. As already discussed 
in the previous lectures, we can write down a semi universal unfolding/deformation 
by choosing a monomial basis for the Milnor- or the Tjurina algebra, respectively. 

Let us do this for the $A_3$-hypersurface singularity $f = x^4 - y\cdot z$. 

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
f = x^4 - y*z
J = ideal(R, minors(jacobi_matrix(f), 1))

# Let us first manually gather a monomial basis for the Milnor algebra
o = negdegrevlex(gens(R)) # a local ordering on the variables of R
leadJ = leading_ideal(J, ordering=o)

Now we can manually gather those monomials in the monomial diagram which are not in the leading ideal 
of the jacobian ideal. In this case, we eventually find $1, x, x^2$.

To automatize this, let us write a function.

In [ ]:
function kbase_local(I::MPolyIdeal)
    R = base_ring(I)
    
    o = negdegrevlex(gens(R))
    leadI = leading_ideal(I, ordering=o)
    
    mon_base = elem_type(R)[]
    d = 0
    new_mons = [x for x in Oscar.all_monomials(R, d) if !(x in leadI)]
    while length(new_mons) > 0 
        mon_base = vcat(mon_base, new_mons)
        d = d+1
        new_mons = [x for x in Oscar.all_monomials(R, d) if !(x in leadI)]
    end
    return mon_base
end

Does it reproduce the same result? Let's check.

In [ ]:
kbase_local(J)

We can now write down a semi-universal unfolding of $f$. First, we need to introduce new 
variables, the deformation parameters.

In [ ]:
function semi_universal_unfolding_local(f::MPolyElem)
    R = parent(f)
    J = ideal(R, minors(jacobi_matrix(f), 1))
    mon_base = kbase_local(J)
    mu = length(mon_base) # The Milnor number
    n = ngens(R)
    
    # set up the new ring with the deformation parameters.
    Rext, v = PolynomialRing(coefficient_ring(R), vcat(symbols(R), [Symbol("u$i") for i in 1:mu]))
    u = gens(Rext)[n+1:end]
    
    # set up the usual map R → Rext
    inc = hom(R, Rext, gens(Rext)[1:n])
    F = inc(f)
    F = F + sum([t*inc(x) for (t, x) in zip(u, mon_base)])
    return (F, gens(Rext)[1:n], gens(Rext)[n+1:end])
end

Let us check that it gives the expected result.

In [ ]:
F, X, U = semi_universal_unfolding_local(f)

## The discriminant of an IHS

Let $F(x; t) \colon (\mathbb C^n, 0) \times (\mathbb C^\mu,0) \to (\mathbb C,0)$ be the semi-universal unfolding 
of an isolated hypersurface singularity $f = f_0 = F(x; 0)$. The *discriminant* is defined as the set of 
parameters $t \in \mathbb C^\mu$ for which the fiber $X_t =  f^{-1}(\{0\})$ is singular. Can we compute that locus?

First, we need a method to compute the "relative Jacobian matrix" where we take the derivatives only with respect to the variables, not the parameters. We can pass the variables as a second argument and simply add that method 
to the existing `jacobi_matrix` function. 

Note that julia distinguishes between different *methods* of the same 
*function* according to the *signature*, i.e. the number of arguments and their types.

In [ ]:
function Oscar.jacobi_matrix(f::MPolyElem, x::Vector{T}) where {T<:MPolyElem}
    R = parent(f)
    all(x->(x in gens(R)), x) || error("The given elements are not all ring variables")
    n = length(x)
    A = zero(MatrixSpace(R, n, 1))
    for i in 1:n
      A[i, 1] = derivative(f, x[i])
    end
    return A
end

Having the above method, we can manually compute the equation for the discriminant $\Delta$ using elimination 
orderings for the projection to the parameter space, cf. yesterday's lectures.

In [ ]:
RU = parent(F) # The ring of the ambient space of the deformation 
JF = ideal(RU, minors(jacobi_matrix(F, X), 1)) + ideal(RU, F) # The relative Tjurina ideal 

# We need an elemination ordering for the projection to the deformation base
o = degrevlex(gens(RU)[1:3])*negdegrevlex(gens(RU)[4:5])

# compute a standard basis w.r.t this elimination ordering
SB = std_basis(JF, o)

# collect only those elements which are contained in the subring of the base
disc_eqns = [f for f in SB if all(v->!(divides(leading_monomial(f, o), v))[1], gens(RU)[1:3])]

The specialists among us may recognize that this is indeed the formula for the discriminant 
of a (depressed) polynomial of degree $4$. It is also known as the "swallow tail", a famous 
singularity. 

We can investigate the singular locus of $\Delta$. 

In [ ]:
dec = primary_decomposition(singular_slocus(ideal(RU, disc_eqns)))

What a mess! But luckily, we already picked out some special points on $\Delta$ for you; 
namely 

 * $t_1 = (0,0,-1)$ on the regular locus of $\Delta$; 
 * $t_2 = (-3, 8, -6)$ on the second component above; 
 * $t_3 = (1, 0, -2)$ on the first component. 
 
Let's check that these points really lay on them. 

In [ ]:
delta = disc_eqns[1]
t1 = [0,0,0, 0,0, -1]
@show evaluate(delta, t1)
@show all(g->iszero(evaluate(g, t1)), gens(dec[2][1]))
@show all(g->iszero(evaluate(g, t1)), gens(dec[1][1]))

In [ ]:
t2 = [0,0,0, -3, 8, -6]
@show evaluate(delta, t2)
@show all(g->iszero(evaluate(g, t2)), gens(dec[2][1]))
@show all(g->iszero(evaluate(g, t2)), gens(dec[1][1]))

In [ ]:
t3 = [0,0,0, 1, 0, -2]
@show evaluate(delta, t3)
@show all(g->iszero(evaluate(g, t3)), gens(dec[2][1]))
@show all(g->iszero(evaluate(g, t3)), gens(dec[1][1]))

Let us analyze the singularities in the fibers for either one of the three parameters. First $t_1$:

In [ ]:
f1 = evaluate(F, R.([x,y,z, 0, 0, -1]))
radical(singular_slocus(ideal(R, f1)))

So $f_1 = F(x,y,z; t_1)$ has an isolated singularity at the origin. Which one is it?

In [ ]:
classify(f1)

In fact, the whole regular stratum of $\Delta$ is characterized by the fibers having one single $A_1$-singularity.

Let us repeat the same with $t_2$.

In [ ]:
f2 = evaluate(F, R.([x,y,z, -3, 8, -6]))
radical(singular_slocus(ideal(R, f2)))

The `classify` command can only analyze singularities at the origin (because it is using local orderings in the background) and, hence, we first have to shift our function.

In [ ]:
shift = hom(R, R, [x+1, y, z])
classify(shift(f2))

So we see that on this stratum of $\Delta$ the fibers have precisely one $A_2$ singularity.

In [ ]:
f3 = evaluate(F, R.([x,y,z, 1, 0, -2]))
radical(singular_slocus(ideal(R, f3)))

Here, we have two singularities and therefore need two shifts.

In [ ]:
shift1 = hom(R, R, [x+1, y, z])
classify(shift1(f3))

In [ ]:
shift2 = hom(R, R, [x-1, y, z])
classify(shift2(f3))

We conclude that on the third stratum, the fibers have two singularities of type $A_1$.

## Deformations of complete intersections

We use the $S_5$-singularity from before.

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
I = ideal(R, [x*y, x^2 + y^2 + z^2])

The normal module is defined as $N = \mathrm{Hom}_R(I, R/I)$. OSCAR knows how to handle `Hom`-modules, 
but we need to convert all the participants into modules, first.

In [ ]:
F = FreeMod(R, 5)
F[2]

In [ ]:
F1 = FreeMod(R, 1) # A free module over R of rank 1
Imod, _ = sub(F1, [g*F1[1] for g in gens(I)]) # the submodule of F1 generated by I
RmodI, _ = quo(F1, Imod) # the quotient by that submodule

In [ ]:
RmodI

In [ ]:
N, vec_to_hom = hom(Imod, RmodI)

The last command is a bit subtle. It returns a pair `(N, vec_to_hom)` consisting of the actual 
module `N` and a map `vec_to_hom`. While often we can discard any extra maps returned by 
constructors, we will need this one in the following. 

What does `vec_to_hom` do? Well, `N` is constructed as a `Hom`-module. A priori, this is just another 
finitely generated module, but also every element $v \in N$ can be interpreted as a module homomorphism 
$ I \to R/I$. The returned map `vec_to_hom` allows us to do that interpretation on the computer side 
so that we really get an honest morphism of modules.

In [ ]:
v = x*N[1] # the first generator of N
typeof(v)

We can try to apply `v` to an element of `Imod` directly, but this will fail.

In [ ]:
g = Imod[2] # the second generator of I
v(g) # This will complain!

Converting `v` into a homomorphism makes everything work.

In [ ]:
phi = vec_to_hom(v)
@show phi
phi(g)

We proceed with the construction of the $T^1(S_5)$. We need to divide by the image of the jacobian matrix.

In [ ]:
f = gens(I)
Df = jacobi_matrix(f)

Now we need the complicated map 
$\theta \colon R^n \to N, \quad (v_1,\dots,v_n) \mapsto \left( \sum_j a_j \cdot f_j \mapsto \sum_{i,j} v_i \frac{\partial f_j}{\partial x_i} a_j \right)$. 
For this we need the `vec_to_hom` map from before and take *preimages* of elements along this map. 
Eventually, we arrive at the following, complicated expression.

In [ ]:
Fn = FreeMod(R, ngens(R))
theta = hom(Fn, N, [preimage(vec_to_hom, hom(Imod, RmodI, [Df[i,j]*RmodI[1] for j in 1:ngens(Imod)])) for i in 1:ngens(R)])
S, _ = image(theta)
T1, _ = quo(N, S)
kbase(T1)

## Versal deformations of arbitrary isolated singularities

Singular (and hence Oscar) knows how to compute the semi-universal deformation of an arbitrary 
singularity according to Grauert's theorem. Let us demonstrate this in Pinkham's example (cf. the 
first lecture on determinantal singularities by Prof. Maria Ruas).

In [ ]:
R, x = PolynomialRing(QQ, "x" => 1:5)
M1 = R[x[1] x[2] x[3] x[4]; x[2] x[3] x[4] x[5]]

In [ ]:
I = ideal(R, minors(M1, 2))

In [ ]:
inc, p = versal_unfolding(I)

Apart from the verbose Singular output during the computation, this returns us a pair of two maps: the inclusion of 
the central fiber `inc` into the family and the projection `p` down to the parameter space. 
The domains and codomains of these maps are 'space germs'.

In [ ]:
domain(inc)

In [ ]:
domain(p)

In [ ]:
codomain(p)

In [ ]:
primary_decomposition(modulus(OO(codomain(p))))